In [1]:
from train_and_test import *
from collections import defaultdict

In [2]:
import warnings

# To ignore all warnings
warnings.filterwarnings("ignore")

## Training

In [26]:
df_data = get_train_data("20231025", "20231213", test_stations)

2023-10-25 00:00:00
2023-10-26 00:00:00
2023-10-27 00:00:00
2023-10-28 00:00:00
2023-10-29 00:00:00
2023-10-30 00:00:00
2023-10-31 00:00:00
2023-11-01 00:00:00
2023-11-02 00:00:00
2023-11-03 00:00:00
2023-11-04 00:00:00
2023-11-05 00:00:00
2023-11-06 00:00:00
2023-11-07 00:00:00
2023-11-08 00:00:00
2023-11-09 00:00:00
2023-11-10 00:00:00
2023-11-11 00:00:00
2023-11-12 00:00:00
2023-11-13 00:00:00
2023-11-14 00:00:00
2023-11-15 00:00:00
2023-11-16 00:00:00
2023-11-17 00:00:00
2023-11-18 00:00:00
2023-11-19 00:00:00
2023-11-20 00:00:00
2023-11-21 00:00:00
2023-11-22 00:00:00
2023-11-23 00:00:00
2023-11-24 00:00:00
2023-11-25 00:00:00
2023-11-26 00:00:00
2023-11-27 00:00:00
2023-11-28 00:00:00
2023-11-29 00:00:00
2023-12-02 00:00:00
2023-12-03 00:00:00
2023-12-04 00:00:00
2023-12-05 00:00:00
2023-12-06 00:00:00
2023-12-07 00:00:00
2023-12-08 00:00:00
2023-12-09 00:00:00
2023-12-10 00:00:00
2023-12-11 00:00:00
2023-12-12 00:00:00
2023-12-13 00:00:00


In [27]:
x_data, features = feature_selection(df_data, 0, 0, 20)

In [28]:
df_feat = pd.DataFrame(np.vstack(x_data), columns=features)

In [29]:
# add features
def add_features(df):
    df['month'] = [d.month for d in df['date']]
    df['day'] = [d.day for d in df['date']]
    df['time_str'] = [str(time).split(' ')[1] for time in df['date']]
    df['date_str'] = [d.strftime('%Y-%m-%d') for d in df['date']]
    df['isHoliday'] = [1 if df['sat'][i] == 1 or df['sun'][i] == 1 else 0 for i in range(len(df))]
    df['isWeekday'] = [1 if df['isHoliday'][i] == 0 else 0 for i in range(len(df))]
    df['weekday'] = [str(df['date'][i].weekday()) for i in range(len(df))]
    df['midnight'] = [1 if hr < 6 else 0 for hr in df['hour']]
    df['morning'] = [1 if hr >= 6 and hr < 12 else 0 for hr in df['hour']]
    df['afternoon'] = [1 if hr >= 12 and hr < 18 else 0 for hr in df['hour']]
    df['night'] = [1 if hr >= 18 and hr < 24 else 0 for hr in df['hour']]
    df['area'] = [demo[station]["sarea"] for station in df["station_id"]]
    return df

In [30]:
df_feat = add_features(df_feat)

In [8]:
# add 7 days ago
df_feat['history'] = list(df_feat['sbi'][72 * 7:]) + list(df_feat['sbi'][:7 * 72])

In [31]:
df_feat.head()

,station_id,date,lng,lat,hour,minute,sbi,tot,mon,tue,...,time_str,date_str,isHoliday,isWeekday,weekday,midnight,morning,afternoon,night,area
0,500101001,2023-10-25 00:00:00,121.5436,25.02605,0,0,17,28,0,0,...,00:00:00,2023-10-25,0,1,2,1,0,0,0,大安區
1,500101001,2023-10-25 00:20:00,121.5436,25.02605,0,20,11,28,0,0,...,00:20:00,2023-10-25,0,1,2,1,0,0,0,大安區
2,500101001,2023-10-25 00:40:00,121.5436,25.02605,0,40,8,28,0,0,...,00:40:00,2023-10-25,0,1,2,1,0,0,0,大安區
3,500101001,2023-10-25 01:00:00,121.5436,25.02605,1,0,7,28,0,0,...,01:00:00,2023-10-25,0,1,2,1,0,0,0,大安區
4,500101001,2023-10-25 01:20:00,121.5436,25.02605,1,20,8,28,0,0,...,01:20:00,2023-10-25,0,1,2,1,0,0,0,大安區


In [32]:
df_train = df_feat.copy()

In [33]:
dic_tot = get_all_tot('20231002')

In [34]:
def add_FE_week_time(df_train):
    df_train['full_week_time'] = [1 if df_train['sbi'][i] > 2/3 * dic_tot[df_train['station_id'][i]] else 0 for i in range(len(df_train))]
    df_train['empty_week_time'] = [1 if df_train['sbi'][i] < 1/3 * dic_tot[df_train['station_id'][i]] else 0 for i in range(len(df_train))]

    full = df_train.groupby(['time_str', 'station_id', 'weekday'])['full_week_time'].sum().reset_index()
    empty = df_train.groupby(['time_str', 'station_id', 'weekday'])['empty_week_time'].sum().reset_index()

    df_train = pd.merge(df_train.drop(['full_week_time', 'empty_week_time'], axis=1), full, on=['time_str', 'station_id', 'weekday'])
    df_train = pd.merge(df_train, empty, on=['time_str', 'station_id', 'weekday'])
    return df_train

In [35]:
def add_FE_time(df_train, time_span, full_thrhld, emp_thrhld):
    df_train['full_time'] = [1 if df_train['sbi'][i] > 2/3 * dic_tot[df_train['station_id'][i]] else 0 for i in range(len(df_train))]
    df_train['empty_time'] = [1 if df_train['sbi'][i] < 1/3 * dic_tot[df_train['station_id'][i]] else 0 for i in range(len(df_train))]

    full = df_train.groupby(['time_str', 'station_id'])['full_time'].sum().reset_index()
    empty = df_train.groupby(['time_str', 'station_id'])['empty_time'].sum().reset_index()

    df_train = pd.merge(df_train.drop(['full_time', 'empty_time'], axis=1), full, on=['time_str', 'station_id'])
    df_train = pd.merge(df_train, empty, on=['time_str', 'station_id'])
    
    df_train['full_time'] = df_train['full_time'] > time_span * full_thrhld
    df_train['empty_time'] = df_train['empty_time'] > time_span * emp_thrhld
    return df_train

In [13]:
# df_train = add_FE_week_time(df_train)
df_train = add_FE_time(df_train, 35, 0.7, 0.01)
df_train = df_train.sort_values(by=['station_id', 'date']).reset_index(drop=True)

In [39]:
df_train.head()

,station_id,date,lng,lat,hour,minute,sbi,tot,mon,tue,...,date_str,isHoliday,isWeekday,weekday,midnight,morning,afternoon,night,area,history
0,500101001,2023-10-25 00:00:00,121.5436,25.02605,0,0,17,28,0,0,...,2023-10-25,0,1,2,1,0,0,0,大安區,17
1,500101001,2023-10-25 00:20:00,121.5436,25.02605,0,20,11,28,0,0,...,2023-10-25,0,1,2,1,0,0,0,大安區,17
2,500101001,2023-10-25 00:40:00,121.5436,25.02605,0,40,8,28,0,0,...,2023-10-25,0,1,2,1,0,0,0,大安區,16
3,500101001,2023-10-25 01:00:00,121.5436,25.02605,1,0,7,28,0,0,...,2023-10-25,0,1,2,1,0,0,0,大安區,16
4,500101001,2023-10-25 01:20:00,121.5436,25.02605,1,20,8,28,0,0,...,2023-10-25,0,1,2,1,0,0,0,大安區,17


In [15]:
# with open('../data/每日降雨.json', 'r') as file:
#     prcp_data = json.load(file)
# taipei = prcp_data['cwaopendata']['resources']['resource']['data']['surfaceObs']['location'][3]['stationObsTimes']['stationObsTime']
# prcp_dic = {}
# for obj in taipei:
#     key = obj['Date']
#     val = obj['weatherElements']['Precipitation']
#     if val == 'T':
#         prcp_dic[key] = 0
#     else:
#         if float(val) > 0:
#             prcp_dic[key] = 1
#         else:
#             prcp_dic[key] = 0

In [17]:
# df_train['prcp_simple'] = [prcp_dic[d] for d in df_train['date_str']]

In [57]:
# with open("../data/weather_dic_year.json", 'r') as f:
#     dic_wthr = json.load(f)
# df_train['temp'] = [dic_wthr[f"{df_train['date_str'][i]}_{df_train['hour'][i]}"]['temp'] for i in range(len(df_train))]
# df_train['prcp'] = [dic_wthr[f"{df_train['date_str'][i]}_{df_train['hour'][i]}"]['prcp'] for i in range(len(df_train))]
# df_train['ws'] = [dic_wthr[f"{df_train['date_str'][i]}_{df_train['hour'][i]}"]['ws'] for i in range(len(df_train))]
# df_train['rh'] = [dic_wthr[f"{df_train['date_str'][i]}_{df_train['hour'][i]}"]['rh'] for i in range(len(df_train))]
# new_prcp = []
# for i in range(len(df_train['prcp'])):
#     if df_train['prcp'][i] == "T":
#         new_prcp.append(0)
#     else:
#         if float(df_train['prcp'][i]) > 0:
#             new_prcp.append(1)
#         else:
#             new_prcp.append(0)
# df_train['new_prcp'] = new_prcp
# df_train.head()

,station_id,date,lng,lat,hour,minute,sbi,tot,mon,tue,...,afternoon,night,area,full,empty,temp,prcp,ws,rh,new_prcp
0,500101001,2023-10-26 00:00:00,121.5436,25.02605,0,0,17,28,0,0,...,0,0,大安區,0,0,17.5,0.0,3.8,98.0,0
1,500101001,2023-10-26 00:20:00,121.5436,25.02605,0,20,11,28,0,0,...,0,0,大安區,0,0,17.5,0.0,3.8,98.0,0
2,500101001,2023-10-26 00:40:00,121.5436,25.02605,0,40,10,28,0,0,...,0,0,大安區,0,0,17.5,0.0,3.8,98.0,0
3,500101001,2023-10-26 01:00:00,121.5436,25.02605,1,0,9,28,0,0,...,0,0,大安區,0,1,16.9,0.0,1.7,87.0,0
4,500101001,2023-10-26 01:20:00,121.5436,25.02605,1,20,9,28,0,0,...,0,0,大安區,0,1,16.9,0.0,1.7,87.0,0


In [19]:
# read from csv
# df_feat = pd.read_csv('../data/df_feat.csv')
# df_feat['date'] = pd.to_datetime(df_feat['date'])
# df_feat.head()


### Validation

In [40]:
def sequence_validation(model_name, df_feat, train_days, test_days, selected_feat):
    time_span = (list(df_feat['date'])[-1] - list(df_feat['date'])[0]).days + 1
    Average_Eval = []
    for i in range(time_span - train_days - test_days + 1):
        train_data = df_feat[(df_feat['date'] >= (df_feat['date'][0] + timedelta(days=i))) & (df_feat['date'] < df_feat['date'][0] + timedelta(days=train_days + i))].reset_index(drop = True)
        test_data = df_feat[(df_feat['date'] >= df_feat['date'][0] + timedelta(days=train_days + i)) & (df_feat['date'] < df_feat['date'][0] + timedelta(days=train_days + test_days + i))].reset_index(drop = True)
        
        print(f"Train Period:{list(train_data['date'])[0]} to {list(train_data['date'])[-1]}")
        print(f"Test Period:{list(test_data['date'])[0]} to {list(test_data['date'])[-1]}")
        
        train_data = add_FE_week_time(train_data)
        train_data = add_FE_time(train_data, train_days, 0.7, 0.7)
        train_data = train_data.sort_values(by=['station_id', 'date']).reset_index(drop=True)
        
        empty_mapping = train_data.set_index(['station_id', 'time_str', 'weekday'])['empty_week_time'].to_dict()
        full_mapping = train_data.set_index(['station_id', 'time_str', 'weekday'])['full_week_time'].to_dict()
        test_data['empty_week_time'] = [empty_mapping[(test_data['station_id'][i], test_data['time_str'][i], test_data['weekday'][i])] for i in range(len(test_data))]
        test_data['full_week_time'] = [full_mapping[(test_data['station_id'][i], test_data['time_str'][i], test_data['weekday'][i])] for i in range(len(test_data))]
        
        empty_mapping = train_data.set_index(['station_id', 'time_str'])['empty_time'].to_dict()
        full_mapping = train_data.set_index(['station_id', 'time_str'])['full_time'].to_dict()
        test_data['empty_time'] = [empty_mapping[(test_data['station_id'][i], test_data['time_str'][i])] for i in range(len(test_data))]
        test_data['full_time'] = [full_mapping[(test_data['station_id'][i], test_data['time_str'][i])] for i in range(len(test_data))]
        
        x_train, y_train = train_data[selected_feat], train_data['sbi']
        x_test, y_test, slot_test = test_data[selected_feat], list(test_data['sbi']), list(test_data['tot'])

        E_val = []
        model = model_selection(x_train.values, y_train.values, model_name)
        if model_name == "svr":
            pred, p_acc, _ =  predict([1000 for _ in range(len(x_test))], list(x_test.values), model)
        else:
            pred = model.predict(x_test.values)
        for j in range(len(pred)):
            real_pred = min(max(0, pred[j]), slot_test[j])
            score = 3 * abs((y_test[j] -  real_pred) / slot_test[j]) * (abs(y_test[j] / slot_test[j] - 1 / 3) + abs(y_test[j] / slot_test[j] - 2 / 3))
            # if score > 2:
            #     print(x_test.values[j], y_test[j], real_pred)
            E_val.append(score)
        print(f"Validation for Fold {i} = {np.average(E_val)}\n")
        Average_Eval.append(np.average(E_val))
        # return E_val
    print(f"\nOverall Performance = {np.average(Average_Eval)}")   
    return np.average(Average_Eval)

In [37]:
selected_feat = ['lng', 'lat', 'hour', 'minute', 'day', 'isHoliday']
sequence_validation('rf', select_data(df_train, "20231026", "20231213"), 35, 7, selected_feat)

Train Period:2023-10-26 00:00:00 to 2023-11-15 23:40:00
Test Period:2023-11-16 00:00:00 to 2023-11-22 23:40:00
Validation for Fold 0 = 0.41075291004399744



KeyboardInterrupt: 

In [41]:
selected_feat = ['lng', 'lat', 'hour', 'full_week_time', 'empty_week_time', 'empty_time', 'full_time']
sequence_validation('rf', select_data(df_train, "20231026", "20231213"), 35, 7, selected_feat)

Train Period:2023-10-26 00:00:00 to 2023-11-29 23:40:00
Test Period:2023-11-30 00:00:00 to 2023-12-06 23:40:00
Validation for Fold 0 = 0.400615843510557

Train Period:2023-10-27 00:00:00 to 2023-11-30 23:40:00
Test Period:2023-12-01 00:00:00 to 2023-12-07 23:40:00
Validation for Fold 1 = 0.3981686475525449

Train Period:2023-10-28 00:00:00 to 2023-12-01 23:40:00
Test Period:2023-12-02 00:00:00 to 2023-12-08 23:40:00
Validation for Fold 2 = 0.3969895935215074

Train Period:2023-10-29 00:00:00 to 2023-12-02 23:40:00
Test Period:2023-12-03 00:00:00 to 2023-12-09 23:40:00
Validation for Fold 3 = 0.40817459356354086

Train Period:2023-10-30 00:00:00 to 2023-12-03 23:40:00
Test Period:2023-12-04 00:00:00 to 2023-12-10 23:40:00
Validation for Fold 4 = 0.40634690333911283

Train Period:2023-10-31 00:00:00 to 2023-12-04 23:40:00
Test Period:2023-12-05 00:00:00 to 2023-12-11 23:40:00
Validation for Fold 5 = 0.4042801146675872


Overall Performance = 0.4024292826924751


0.4024292826924751

### oob error

In [48]:
selected_feat = ['lng', 'lat', 'hour', 'minute', 'day']
train_data = select_data(df_train, "20231101", "20231114")
x_train, y_train = train_data[selected_feat], train_data["empty"]
model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1,oob_score=True)
model.fit(x_train, y_train)
print(model.oob_score_)

0.9084024234693877


In [33]:
selected_feat = ['lng', 'lat','minute', 'day', 'hour', "area_code"]
train_data = select_data(df_train, "20231101", "20231107")
x_train, y_train = train_data[selected_feat], train_data["sbi"]
model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1,oob_score=True)
model.fit(x_train, y_train)
print(model.oob_score_)

0.8836496119274625


### Aggregation Test

In [23]:
def calculate_score(pred, y_test, slot_test):
    return np.average([3 * abs((y_test[j] -  pred[j]) / slot_test[j]) * (abs(y_test[j] / slot_test[j] - 1 / 3) + abs(y_test[j] / slot_test[j] - 2 / 3)) for j in range(len(pred))])

In [21]:
def aggregation_test(model_list, train_days, test_days, df_feat):
    
    time_span = (list(df_feat['date'])[-1] - list(df_feat['date'])[0]).days + 1
    Average_Eval = [[] for _ in range(11)]
    for i in range(time_span - train_days - test_days + 1):
        train_data = df_feat[(df_feat['date'] >= (df_feat['date'][0] + timedelta(days=i))) & (df_feat['date'] < df_feat['date'][0] + timedelta(days=train_days + i))].reset_index(drop = True)
        test_data = df_feat[(df_feat['date'] >= df_feat['date'][0] + timedelta(days=train_days + i)) & (df_feat['date'] < df_feat['date'][0] + timedelta(days=train_days + test_days + i))].reset_index(drop = True)
        
        train_data = add_FE_week_time(train_data)
        train_data = add_FE_time(train_data, train_days, 0.5, 0.5)
        train_data = train_data.sort_values(by=['station_id', 'date']).reset_index(drop=True)
        
        empty_mapping = train_data.set_index(['station_id', 'time_str', 'weekday'])['empty_week_time'].to_dict()
        full_mapping = train_data.set_index(['station_id', 'time_str', 'weekday'])['full_week_time'].to_dict()
        test_data['empty_week_time'] = [empty_mapping[(test_data['station_id'][i], test_data['time_str'][i], test_data['weekday'][i])] for i in range(len(test_data))]
        test_data['full_week_time'] = [full_mapping[(test_data['station_id'][i], test_data['time_str'][i], test_data['weekday'][i])] for i in range(len(test_data))]
        
        empty_mapping = train_data.set_index(['station_id', 'time_str'])['empty_time'].to_dict()
        full_mapping = train_data.set_index(['station_id', 'time_str'])['full_time'].to_dict()
        test_data['empty_time'] = [empty_mapping[(test_data['station_id'][i], test_data['time_str'][i])] for i in range(len(test_data))]
        test_data['full_time'] = [full_mapping[(test_data['station_id'][i], test_data['time_str'][i])] for i in range(len(test_data))]
        
        
        all_pred = []
        for cur_model in model_list:
            x_train, y_train = train_data[cur_model["selected_feat"]], train_data['sbi']
            x_test, y_test, slot_test = test_data[cur_model["selected_feat"]], list(test_data['sbi']), list(test_data['tot'])

            model = model_selection(x_train.values, y_train.values, cur_model["name"])
            if cur_model["name"] == "svr":
                pred, p_acc, _ =  predict([1000 for _ in range(len(x_test))], list(x_test.values), model)
            else:
                pred = model.predict(x_test.values)
            
            real_pred = [min(max(0, pred[j]), slot_test[j]) for j in range(len(pred))]
            all_pred.append(real_pred)
            score = calculate_score(pred, y_test, slot_test)
            # print(f"{cur_model['name']}: {(score)}")
        
        for j in np.arange(11):
            w = [j/10, (10-j)/10]
            agg_list = [np.dot(col, w) / sum(w) for col in zip(*all_pred)]
            score = calculate_score(agg_list, y_test, slot_test)
            # print(f"Aggregation for Fold {i}, w: {round(j, 1), round(1-j, 1)} = {score}\n")
            Average_Eval[j].append(score)
        # return E_val
    lowest_Eval = 1
    best_i = 0.1
    for i in range(11):
        print(f"\n Performance {i, 10-i} = {np.average(Average_Eval[i])}")
        if np.average(Average_Eval[i]) < lowest_Eval:
            lowest_Eval = np.average(Average_Eval[i])
            best_i = (i, 10-i)
    return best_i, lowest_Eval
    

In [42]:
aggregation_test([
    {"name": 'rf', "selected_feat": ['lng', 'lat', 'hour', 'full_week_time', 'empty_week_time', 'empty_time', 'full_time']},
    # {"name": 'adaboost', "selected_feat": ['lat', 'lng', 'hour', 'minute', 'day', 'isHoliday']},
    {"name": 'rf', "selected_feat": ['lng', 'lat', 'hour', 'minute', 'day', 'isHoliday']},
    # {"name": 'svr', "selected_feat": ['lat', 'lng', 'hour', 'minute', 'day', 'isHoliday', 'full_week_time', 'empty_week_time', 'empty_time', 'full_time']},
    ], 
            35, 7, select_data(df_train, "20231026", "20231213"))


 Performance (0, 10) = 0.4144215286058954

 Performance (1, 9) = 0.40610705740625136

 Performance (2, 8) = 0.399978425512483

 Performance (3, 7) = 0.39568457199880197

 Performance (4, 6) = 0.39290648568697306

 Performance (5, 5) = 0.3914719159350133

 Performance (6, 4) = 0.3913314449210407

 Performance (7, 3) = 0.3922997861179495

 Performance (8, 2) = 0.3943981966556818

 Performance (9, 1) = 0.3977110470057035

 Performance (10, 0) = 0.40241011085145506


((6, 4), 0.3913314449210407)